# 1.Load & Clean Tabular Data


In [9]:
import pandas as pd
import numpy as np

train = pd.read_excel("C:/Users/Sathwika/OneDrive/Desktop/Satellite_property_valuation/Data/Raw/train(1).xlsx")
test  = pd.read_excel("C:/Users/Sathwika/OneDrive/Desktop/Satellite_property_valuation/Data/Raw/test2.xlsx")

print(train.shape, test.shape)
train.head()

train = train.drop_duplicates()

# Handle missing values SAFELY
TARGET_COL = "price"

# Select numeric columns EXCLUDING target
num_cols = train.select_dtypes(include=np.number).columns.tolist()
num_cols.remove(TARGET_COL)   # IMPORTANT

for col in num_cols:
    median_val = train[col].median()
    train[col] = train[col].fillna(median_val)
    
    # Fill test only if column exists
    if col in test.columns:
        test[col] = test[col].fillna(median_val)


# Target transformation (TRAIN ONLY)

train["log_price"] = np.log1p(train[TARGET_COL])


# Sanity checks


print("price in train:", "price" in train.columns)
print("price in test :", "price" in test.columns)
print("\nMissing values (train):")
print(train.isna().sum().sort_values(ascending=False).head())
print("\nMissing values (test):")
print(test.isna().sum().sort_values(ascending=False).head())

(16209, 21) (5404, 20)
price in train: True
price in test : False

Missing values (train):
id           0
date         0
price        0
bedrooms     0
bathrooms    0
dtype: int64

Missing values (test):
id             0
date           0
bedrooms       0
bathrooms      0
sqft_living    0
dtype: int64


# 2.Feature Engineering (Ratio-Based Features)


In [10]:
train["house_density_ratio"] = train["sqft_living"] / train["sqft_living15"]
test["house_density_ratio"]  = test["sqft_living"] / test["sqft_living15"]

train["lot_density_ratio"] = train["sqft_lot"] / train["sqft_lot15"]
test["lot_density_ratio"]  = test["sqft_lot"] / test["sqft_lot15"]

train["basement_ratio"] = train["sqft_basement"] / train["sqft_living"]
test["basement_ratio"]  = test["sqft_basement"] / test["sqft_living"]

train.replace([np.inf, -np.inf], 0, inplace=True)
test.replace([np.inf, -np.inf], 0, inplace=True)


# 3.Save Processed Data

In [11]:
train.to_csv(
    "C:/Users/Sathwika/OneDrive/Desktop/Satellite_property_valuation/Data/train_processed.csv",
    index=False
)

test.to_csv(
    "C:/Users/Sathwika/OneDrive/Desktop/Satellite_property_valuation/Data/test_processed.csv",
    index=False
)
